<a href="https://colab.research.google.com/github/Hafeeplay/razor.io/blob/main/Sp2try4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
! pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf
from gensim.models import Word2Vec
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [20]:
# Load and preprocess the data
sentences = []
with open('/content/drive/MyDrive/Specialproject2/amazon_cells_labelled.txt') as f:
    lines = f.readlines()
    for line in lines:
        sentences.append(line.split('\t')[0].split())

# Train the Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Save the model
model.save("word2vec.model")

# Load the Word2Vec model
w2v_model = Word2Vec.load("word2vec.model")


In [36]:
# Load the dataset
df = pd.read_csv("/content/drive/MyDrive/Specialproject2/depression_dataset_reddit_cleaned.csv")
sentences = df['clean_text'] #.values.tolist()
labels = df['is_depression'] #.values.tolist()

In [37]:

# Preprocess the data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=100)

In [38]:
# Create the embedding matrix using the Word2Vec model
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    if word in w2v_model.wv.index_to_key:
        embedding_matrix[i] = w2v_model.wv[word]

In [39]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [40]:
# # Pad the sequences to the same length
# max_length = 190
# train_padded = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post')
# test_padded = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post')

In [41]:
# Build the model
model = Sequential()
model.add(Embedding(len(word_index) + 1, 100, weights=[embedding_matrix], input_length=100, trainable=False))
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(1, activation='sigmoid'))

In [43]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [44]:
# Train the model
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
49/49 [==============================] - 58s 1s/step - loss: 0.4584 - accuracy: 0.7681 - val_loss: 0.3372 - val_accuracy: 0.8565
Epoch 2/10
49/49 [==============================] - 52s 1s/step - loss: 0.3451 - accuracy: 0.8512 - val_loss: 0.3256 - val_accuracy: 0.8681
Epoch 3/10
49/49 [==============================] - 56s 1s/step - loss: 0.3309 - accuracy: 0.8587 - val_loss: 0.3495 - val_accuracy: 0.8474
Epoch 4/10
49/49 [==============================] - 53s 1s/step - loss: 0.3327 - accuracy: 0.8596 - val_loss: 0.3242 - val_accuracy: 0.8681
Epoch 5/10
49/49 [==============================] - 52s 1s/step - loss: 0.3215 - accuracy: 0.8619 - val_loss: 0.3270 - val_accuracy: 0.8630
Epoch 6/10
49/49 [==============================] - 53s 1s/step - loss: 0.3222 - accuracy: 0.8634 - val_loss: 0.3125 - val_accuracy: 0.8746
Epoch 7/10
49/49 [==============================] - 53s 1s/step - loss: 0.3188 - accuracy: 0.8655 - val_loss: 0.3169 - val_accuracy: 0.8694
Epoch 8/10
49/49 [==

In [45]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy)

49/49 [==============================] - 2s 48ms/step - loss: 0.3222 - accuracy: 0.8655
Test accuracy: 0.8655462265014648
